<a href="https://colab.research.google.com/github/mykingdom216/hotel-booking-app/blob/main/Hotel_Booking_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import re
from datetime import datetime, date, timedelta

# --- Database Initialization ---
def init_db():
    """
    Initializes the in-memory SQLite database, creates tables,
    and populates the rooms table with sample data.
    """
    conn = sqlite3.connect(':memory:')  # Connect to an in-memory database
    cursor = conn.cursor()

    # Create rooms table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS rooms (
            room_id INTEGER PRIMARY KEY AUTOINCREMENT,
            room_number TEXT UNIQUE NOT NULL,
            room_type TEXT NOT NULL,
            price_per_night REAL NOT NULL
        )
    ''')

    # Create customers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT
        )
    ''')

    # Create bookings table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bookings (
            booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            room_id INTEGER NOT NULL,
            check_in_date TEXT NOT NULL,
            check_out_date TEXT NOT NULL,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (room_id) REFERENCES rooms(room_id)
        )
    ''')

    # Insert sample rooms if not already present
    sample_rooms = [
        ('101', 'Single', 50.00),
        ('102', 'Single', 50.00),
        ('201', 'Double', 80.00),
        ('202', 'Double', 80.00),
        ('301', 'Suite', 150.00)
    ]
    for room_number, room_type, price in sample_rooms:
        try:
            cursor.execute("INSERT INTO rooms (room_number, room_type, price_per_night) VALUES (?, ?, ?)",
                           (room_number, room_type, price))
        except sqlite3.IntegrityError:
            # Room already exists, skip
            pass
    conn.commit()
    return conn

# --- Validation Functions ---
def is_valid_email(email):
    """
    Validates an email address using a regular expression.
    """
    # Simple regex for email validation
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

def is_valid_date(date_str):
    """
    Validates a date string in YYYY-MM-DD format and ensures it's not in the past.
    """
    try:
        parsed_date = datetime.strptime(date_str, '%Y-%m-%d').date()
        if parsed_date < date.today():
            print("Error: Date cannot be in the past.")
            return False
        return True
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")
        return False

def parse_date(date_str):
    """
    Parses a date string into a datetime.date object.
    Assumes date_str has already been validated by is_valid_date.
    """
    return datetime.strptime(date_str, '%Y-%m-%d').date()

# --- Customer Management ---
def add_customer(conn):
    """
    Prompts the user for customer details and adds a new customer to the database.
    Includes validation for email uniqueness.
    """
    cursor = conn.cursor()
    print("\n--- Add New Customer ---")
    name = input("Enter customer name: ").strip()
    if not name:
        print("Customer name cannot be empty.")
        return

    email = input("Enter customer email: ").strip()
    if not is_valid_email(email):
        print("Invalid email format.")
        return

    phone = input("Enter customer phone (optional): ").strip()

    try:
        cursor.execute("INSERT INTO customers (name, email, phone) VALUES (?, ?, ?)",
                       (name, email, phone if phone else None))
        conn.commit()
        print(f"Customer '{name}' added successfully with ID: {cursor.lastrowid}")
    except sqlite3.IntegrityError:
        print(f"Error: A customer with email '{email}' already exists.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def view_customers(conn):
    """
    Retrieves and displays all customer records from the database.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT customer_id, name, email, phone FROM customers")
    customers = cursor.fetchall()

    print("\n--- All Customers ---")
    if not customers:
        print("No customers found.")
        return

    print(f"{'ID':<5} {'Name':<20} {'Email':<30} {'Phone':<15}")
    print("-" * 70)
    for customer in customers:
        print(f"{customer[0]:<5} {customer[1]:<20} {customer[2]:<30} {customer[3] or 'N/A':<15}")

# --- Room Management ---
def view_rooms(conn):
    """
    Retrieves and displays all room records from the database.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT room_id, room_number, room_type, price_per_night FROM rooms")
    rooms = cursor.fetchall()

    print("\n--- All Rooms ---")
    if not rooms:
        print("No rooms found.")
        return

    print(f"{'ID':<5} {'Number':<10} {'Type':<10} {'Price/Night':<15}")
    print("-" * 45)
    for room in rooms:
        print(f"{room[0]:<5} {room[1]:<10} {room[2]:<10} ${room[3]:<14.2f}")


def view_available_rooms(conn):
    """
    Prompts for check-in/out dates and displays rooms available for that period.
    """
    cursor = conn.cursor()
    print("\n--- View Available Rooms ---")
    check_in_str = input("Enter check-in date (YYYY-MM-DD): ").strip()
    if not is_valid_date(check_in_str):
        return

    check_out_str = input("Enter check-out date (YYYY-MM-DD): ").strip()
    if not is_valid_date(check_out_str):
        return

    check_in = parse_date(check_in_str)
    check_out = parse_date(check_out_str)

    if check_out <= check_in:
        print("Error: Check-out date must be after check-in date.")
        return

    # Find rooms that are NOT booked during the requested period
    # This query selects rooms whose booking period does not overlap with the requested period.
    # A room is available if there is no existing booking (B) such that:
    # (B.check_in_date < check_out AND B.check_out_date > check_in)
    cursor.execute(f"""
        SELECT r.room_id, r.room_number, r.room_type, r.price_per_night
        FROM rooms r
        WHERE r.room_id NOT IN (
            SELECT b.room_id
            FROM bookings b
            WHERE (b.check_in_date < '{check_out_str}' AND b.check_out_date > '{check_in_str}')
        )
        ORDER BY r.room_number
    """)
    available_rooms = cursor.fetchall()

    print(f"\n--- Available Rooms from {check_in_str} to {check_out_str} ---")
    if not available_rooms:
        print("No rooms available for the selected dates.")
        return

    print(f"{'ID':<5} {'Number':<10} {'Type':<10} {'Price/Night':<15}")
    print("-" * 45)
    for room in available_rooms:
        print(f"{room[0]:<5} {room[1]:<10} {room[2]:<10} ${room[3]:<14.2f}")

# --- Booking Management ---
def make_booking(conn):
    """
    Prompts for booking details, validates inputs, checks availability,
    and creates a new booking record.
    """
    cursor = conn.cursor()
    print("\n--- Make a New Booking ---")

    try:
        customer_id = int(input("Enter customer ID: ").strip())
    except ValueError:
        print("Invalid customer ID. Please enter a number.")
        return

    # Check if customer exists
    cursor.execute("SELECT customer_id FROM customers WHERE customer_id = ?", (customer_id,))
    if not cursor.fetchone():
        print(f"Error: Customer with ID {customer_id} not found.")
        return

    try:
        room_id = int(input("Enter room ID: ").strip())
    except ValueError:
        print("Invalid room ID. Please enter a number.")
        return

    # Check if room exists
    cursor.execute("SELECT room_id FROM rooms WHERE room_id = ?", (room_id,))
    if not cursor.fetchone():
        print(f"Error: Room with ID {room_id} not found.")
        return

    check_in_str = input("Enter check-in date (YYYY-MM-DD): ").strip()
    if not is_valid_date(check_in_str):
        return

    check_out_str = input("Enter check-out date (YYYY-MM-DD): ").strip()
    if not is_valid_date(check_out_str):
        return

    check_in = parse_date(check_in_str)
    check_out = parse_date(check_out_str)

    if check_out <= check_in:
        print("Error: Check-out date must be after check-in date.")
        return

    # Check for room availability for the specific room and dates
    cursor.execute(f"""
        SELECT COUNT(*)
        FROM bookings
        WHERE room_id = ?
          AND (check_in_date < ? AND check_out_date > ?)
    """, (room_id, check_out_str, check_in_str))

    if cursor.fetchone()[0] > 0:
        print(f"Error: Room {room_id} is not available for the selected dates.")
        return

    try:
        cursor.execute("INSERT INTO bookings (customer_id, room_id, check_in_date, check_out_date) VALUES (?, ?, ?, ?)",
                       (customer_id, room_id, check_in_str, check_out_str))
        conn.commit()
        print(f"Booking created successfully for customer ID {customer_id} in room ID {room_id}.")
    except Exception as e:
        print(f"An unexpected error occurred during booking: {e}")

def view_all_bookings(conn):
    """
    Retrieves and displays all booking records with associated customer and room details.
    """
    cursor = conn.cursor()
    cursor.execute("""
        SELECT
            b.booking_id,
            c.name AS customer_name,
            r.room_number,
            r.room_type,
            b.check_in_date,
            b.check_out_date,
            r.price_per_night
        FROM bookings b
        JOIN customers c ON b.customer_id = c.customer_id
        JOIN rooms r ON b.room_id = r.room_id
        ORDER BY b.check_in_date DESC
    """)
    bookings = cursor.fetchall()

    print("\n--- All Bookings ---")
    if not bookings:
        print("No bookings found.")
        return

    print(f"{'ID':<5} {'Customer Name':<20} {'Room No.':<10} {'Type':<10} {'Check-in':<12} {'Check-out':<12} {'Total Price':<15}")
    print("-" * 90)
    for booking in bookings:
        booking_id, customer_name, room_number, room_type, check_in_str, check_out_str, price_per_night = booking

        check_in = parse_date(check_in_str)
        check_out = parse_date(check_out_str)

        num_nights = (check_out - check_in).days
        total_price = num_nights * price_per_night

        print(f"{booking_id:<5} {customer_name:<20} {room_number:<10} {room_type:<10} {check_in_str:<12} {check_out_str:<12} ${total_price:<14.2f}")

# --- Main Application Loop ---
def main():
    """
    Main function to run the hotel booking application.
    Displays a menu and handles user choices.
    """
    conn = init_db()
    print("Hotel Booking Application started (in-memory database).")

    while True:
        print("\n--- Menu ---")
        print("1. Add New Customer")
        print("2. View All Customers")
        print("3. View All Rooms")
        print("4. View Available Rooms for Dates")
        print("5. Make a Booking")
        print("6. View All Bookings")
        print("7. Exit")

        choice = input("Enter your choice: ").strip()

        if choice == '1':
            add_customer(conn)
        elif choice == '2':
            view_customers(conn)
        elif choice == '3':
            view_rooms(conn)
        elif choice == '4':
            view_available_rooms(conn)
        elif choice == '5':
            make_booking(conn)
        elif choice == '6':
            view_all_bookings(conn)
        elif choice == '7':
            print("Exiting application. Database will be discarded.")
            conn.close()
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()